# etf-dot-com.ipynb

ETF.com からティッカーと基本情報、ファンドフローをスクレイピングするコードを実験するノートブック。

-   Ticker 取得: OK
-   ETF Fundamentals 取得: 一部で loading...や--となっているデータを取得してしまい、欠損扱いになってしまうので、改善必要、。
-   Fund Flow 取得: ticker ひとつについて、取得ロジックを関数化完了。あとは並列処理できるようにする。


[ETF.com](https://www.etf.com/etfanalytics/etf-screener?utm_medium=nav)


In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import io
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import datetime
from dateutil import relativedelta
import time
from concurrent.futures import ProcessPoolExecutor, as_completed
import sqlite3
from pathlib import Path
from etf_dot_com import get_etf_fundamentals, get_tickers_from_db, save_data_to_db
from tqdm import tqdm

ROOT_DIR = Path().cwd().parent
ETF_DIR = ROOT_DIR / "data/ETF"


## Ticker & ETF Fundamentals download


In [ ]:
# --- Ticker download ---

# try:
#     final_df = download_all_tickers(browser_display=False)
#     if not final_df.empty:
#         db_path = ETF_DIR / "ETFDotCom.db"
#         conn = sqlite3.connect(db_path)
#         final_df.to_sql("Ticker", conn, index=False, if_exists="append")
#         conn.close()
#         print(f"\nデータベース{db_path.name}への保存が完了しました。")
#     else:
#         print("\n取得データが空のため、データベースへの保存はスキップされました。")

# except Exception as e:
#     print("\n処理中にエラーが発生しました。")

# --- ETF Fundamentals download ---

db_path = ETF_DIR / "ETFDotCom.db"
ticker_list = get_tickers_from_db(db_path=db_path)

MAX_WORKERS = 5
print(f"\n--- 並列処理を開始します（MAX_WORKERS= {MAX_WORKERS}） ---")
start_time = time.time()
all_data_list = []

with ProcessPoolExecutor(max_workers=MAX_WORKERS) as executor:
    futures = {
        executor.submit(get_etf_fundamentals, ticker): ticker for ticker in ticker_list
    }

    # tqdmを使って進捗を表示
    for future in tqdm(
        as_completed(futures), total=len(ticker_list), desc="スクレイピング進捗"
    ):
        try:
            df = future.result()
            if not df.empty:
                all_data_list.append(df)
        except Exception as e:
            ticker = futures[future]
            print(f"!! ティッカー {ticker} の処理で予期せぬエラー: {e}")

# 3. 結果を結合し、表示
if not all_data_list:
    print("\n\n--- データを一件も取得できませんでした ---")

final_df = pd.concat(all_data_list, ignore_index=True)
print("\n\n--- 全てのティッカーのデータ取得が完了しました ---")
display(final_df)

# 4. 結合したデータをデータベースに保存
save_data_to_db(final_df, db_path, "ETF Fundamentals")

end_time = time.time()
print(f"\n--- 全処理完了 ---")
print(f"実行時間: {end_time - start_time:.2f} 秒")


データベース 'ETFDotCom.db' から本日のティッカーリストを取得しています...
!! データベースからのティッカー取得中にエラー: Execution failed on sql '
        SELECT DISTINCT
            Ticker
        FROM
            Ticker
        WHERE
            DATE(datetime_stored) = '2025-09-29'
    ': no such table: Ticker

--- 並列処理を開始します（MAX_WORKERS= 5） ---


スクレイピング進捗: 0it [00:00, ?it/s]



--- データを一件も取得できませんでした ---


ValueError: No objects to concatenate

In [ ]:
db_path = ETF_DIR / "ETFDotCom.db"
conn = sqlite3.connect(db_path)

df = pd.read_sql("SELECT * FROM 'ETF Fundamentals'", con=conn)


In [ ]:
missing_ticker_list = df[df.eq("loading...").any(axis=1)]["Ticker"].tolist()
MAX_WORKERS = 5
print(f"\n--- 並列処理を開始します（MAX_WORKERS= {MAX_WORKERS}） ---")
start_time = time.time()
all_data_list = []

with ProcessPoolExecutor(max_workers=MAX_WORKERS) as executor:
    futures = {
        executor.submit(get_etf_fundamentals, ticker): ticker
        for ticker in missing_ticker_list
    }

    # tqdmを使って進捗を表示
    for future in tqdm(
        as_completed(futures), total=len(missing_ticker_list), desc="スクレイピング進捗"
    ):
        try:
            df = future.result()
            if not df.empty:
                all_data_list.append(df)
        except Exception as e:
            ticker = futures[future]
            print(f"!! ティッカー {ticker} の処理で予期せぬエラー: {e}")

# 3. 結果を結合し、表示
if not all_data_list:
    print("\n\n--- データを一件も取得できませんでした ---")

final_df_missing = pd.concat(all_data_list, ignore_index=True)
print("\n\n--- 全てのティッカーのデータ取得が完了しました ---")
display(final_df_missing)

# 4. 結合したデータをデータベースに保存
save_data_to_db(final_df_missing, db_path, "ETF Fundamentals")

end_time = time.time()
print(f"\n--- 全処理完了 ---")
print(f"実行時間: {end_time - start_time:.2f} 秒")



--- 並列処理を開始します（MAX_WORKERS= 5） ---


スクレイピング進捗: 100%|██████████| 89/89 [03:11<00:00,  2.15s/it]



--- 全てのティッカーのデータ取得が完了しました ---


,Ticker,Issuer,Inception Date,Expense Ratio,AUM,Index Tracked,Segment,Structure,Asset Class,Category,Focus,Niche,Region,Geography,Index Weighting Methodology,Index Selection Methodology,Segment Benchmark
0,QQQ,Invesco,03/10/99,0.20%,$367.13B,NASDAQ 100 Index,MSCI USA Large Cap,Unit Investment Trust,Equity,Size and Style,Large Cap,Broad-based,North America,U.S.,Market Cap,NASDAQ - Listed,MSCI USA Large Cap
1,SCHM,The Charles Schwab Corp.,01/13/11,0.04%,$12.19B,Dow Jones US Total Stock Market Mid-Cap,MSCI USA Mid Cap,Open-Ended Fund,Equity,Size and Style,Mid Cap,Broad-based,North America,U.S.,Market Cap,Market Cap,MSCI USA Mid Cap
2,MDY,State Street,05/04/95,0.24%,$23.75B,S&P Mid Cap 400,MSCI USA Mid Cap,Open-Ended Fund,Equity,Size and Style,Mid Cap,Broad-based,North America,U.S.,Market Cap,Committee,MSCI USA Mid Cap
3,SHY,"BlackRock, Inc.",07/22/02,0.15%,$24.75B,ICE BofA US Treasury Bond (1-3 Y),ICE BofA US Treasury (1-3 Y),Open-Ended Fund,Fixed Income,"Government, Treasury",Investment Grade,Short-Term,North America,U.S.,Market Value,Market Value,ICE BofA US Treasury (1-3 Y)
4,VOO,Vanguard,09/07/10,0.03%,$751.49B,S&P 500,MSCI USA Large Cap,Open-Ended Fund,Equity,Size and Style,Large Cap,Broad-based,North America,U.S.,Market Cap,Committee,MSCI USA Large Cap
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,PFFL,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--
85,MNRS,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--
86,DMAT,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--
87,MSFY,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--



データベース 'ETFDotCom.db' のテーブル 'ETF Fundamentals' にデータを保存しています...
データベースへの保存が完了しました。（89行）

--- 全処理完了 ---
実行時間: 191.86 秒


In [ ]:
display(final_df_missing[final_df_missing.eq("loading...").any(axis=1)])
df = final_df_missing[final_df_missing.eq("loading...").any(axis=1)]

missing_ticker_list = df[df.eq("loading...").any(axis=1)]["Ticker"].tolist()
MAX_WORKERS = 5
print(f"\n--- 並列処理を開始します（MAX_WORKERS= {MAX_WORKERS}） ---")
start_time = time.time()
all_data_list = []

with ProcessPoolExecutor(max_workers=MAX_WORKERS) as executor:
    futures = {
        executor.submit(get_etf_fundamentals, ticker): ticker
        for ticker in missing_ticker_list
    }

    # tqdmを使って進捗を表示
    for future in tqdm(
        as_completed(futures), total=len(missing_ticker_list), desc="スクレイピング進捗"
    ):
        try:
            df = future.result()
            if not df.empty:
                all_data_list.append(df)
        except Exception as e:
            ticker = futures[future]
            print(f"!! ティッカー {ticker} の処理で予期せぬエラー: {e}")

# 3. 結果を結合し、表示
if not all_data_list:
    print("\n\n--- データを一件も取得できませんでした ---")

final_df_missing = pd.concat(all_data_list, ignore_index=True)
print("\n\n--- 全てのティッカーのデータ取得が完了しました ---")
display(final_df_missing)

# 4. 結合したデータをデータベースに保存
save_data_to_db(final_df_missing, db_path, "ETF Fundamentals")

end_time = time.time()
print(f"\n--- 全処理完了 ---")
print(f"実行時間: {end_time - start_time:.2f} 秒")


,Ticker,Issuer,Inception Date,Expense Ratio,AUM,Index Tracked,Segment,Structure,Asset Class,Category,Focus,Niche,Region,Geography,Index Weighting Methodology,Index Selection Methodology,Segment Benchmark,datetime_created
79,WEED,loading...,loading...,loading...,loading...,--,--,--,--,--,--,--,--,--,--,--,--,2025-09-14 16:45:00



--- 並列処理を開始します（MAX_WORKERS= 5） ---


スクレイピング進捗:   0%|          | 0/1 [00:00<?, ?it/s]

スクレイピング進捗: 100%|██████████| 1/1 [00:06<00:00,  6.46s/it]



--- 全てのティッカーのデータ取得が完了しました ---


,Ticker,Issuer,Inception Date,Expense Ratio,AUM,Index Tracked,Segment,Structure,Asset Class,Category,Focus,Niche,Region,Geography,Index Weighting Methodology,Index Selection Methodology,Segment Benchmark
0,WEED,Roundhill Investments,04/20/22,0.00%,$10.71M,No Underlying Index,--,Open-Ended Fund,Equity,Sector,Theme,Cannabis,Global,Global,Proprietary,Proprietary,--



データベース 'ETFDotCom.db' のテーブル 'ETF Fundamentals' にデータを保存しています...
データベースへの保存が完了しました。（1行）

--- 全処理完了 ---
実行時間: 6.53 秒


### コメント

'--'がついている行がある？


## Experimental Codes


In [ ]:
def scrape_key_value_data(driver, wait, container_id: str) -> dict:
    """
    指定されたコンテナID内のキーと値のペアをスクレイピングして辞書として返す関数
    """
    data_dict = {}
    print(f"ID='{container_id}'のデータを待機しています...")

    # 1. データが表示されるまで待機
    # プレースホルダー（loading...）対策で、2段階の待機を行う
    wait.until(
        lambda d: len(d.find_elements(By.CSS_SELECTOR, f"#{container_id} .inline-flex"))
        > 0
    )
    try:
        wait.until(
            EC.invisibility_of_element_located(
                (
                    By.XPATH,
                    f"//*[@id='{container_id}']//*[contains(text(), 'loading...')]",
                )
            )
        )
    except:
        # loading... が最初から表示されない場合もあるため、タイムアウトは無視して続行
        pass

    print(f"ID='{container_id}'のデータを検出しました。")

    # 2. データを抽出
    container_div = driver.find_element(By.ID, container_id)
    data_rows = container_div.find_elements(By.CLASS_NAME, "inline-flex")
    for row in data_rows:
        children = row.find_elements(By.TAG_NAME, "div")
        if len(children) == 2:
            key = children[0].text
            value = children[1].text
            data_dict[key] = value

    return data_dict


# --- メインの処理 ---
ticker = "VOO"
url = f"https://www.etf.com/{ticker}"
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
wait = WebDriverWait(driver, 10)  # 待機時間を10秒に設定

try:
    driver.get(url)

    # 関数を呼び出して、それぞれのデータを取得
    summary_data = scrape_key_value_data(driver, wait, "summary-data")
    classification_data = scrape_key_value_data(
        driver, wait, "classification-index-data"
    )

    # 取得した2つの辞書を結合
    combined_data = {**summary_data, **classification_data}

    # 結合した辞書から最終的なDataFrameを作成
    df = pd.DataFrame([combined_data])
    df = df.assign(datetime_created=datetime.datetime.now(), Ticker=ticker)

    print("\n--- データフレームの作成に成功しました ---")
    display(df)

except Exception as e:
    print(f"処理中にエラーが発生しました: {e}")

finally:
    driver.quit()


ID='summary-data'のデータを待機しています...
ID='summary-data'のデータを検出しました。
ID='classification-index-data'のデータを待機しています...
ID='classification-index-data'のデータを検出しました。

--- データフレームの作成に成功しました ---


,Issuer,Inception Date,Expense Ratio,AUM,Index Tracked,Segment,Structure,Asset Class,Category,Focus,Niche,Region,Geography,Index Weighting Methodology,Index Selection Methodology,Segment Benchmark,datetime_stored,Ticker
0,Vanguard,09/07/10,0.03%,$751.49B,S&P 500,MSCI USA Large Cap,Open-Ended Fund,Equity,Size and Style,Large Cap,Broad-based,North America,U.S.,Market Cap,Committee,MSCI USA Large Cap,2025-09-12 17:40:03.346187,VOO


### タブ切り替えテスト


In [ ]:
# --- メインの処理 ---
ticker = "VOO"
url = f"https://www.etf.com/{ticker}"
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
wait = WebDriverWait(driver, 10)  # 待機時間を10秒に設定

try:
    # サイトにアクセス
    print(f"{url} にアクセスします...")
    driver.get(url)

    # --- 【ステップ1】 クッキー同意ボタンが表示されたらクリックする ---
    # ポップアップは常に表示されるとは限らないため、try...exceptで囲みます
    try:
        # このXPathは一例です。実際のボタンのテキストや構造に合わせて変更してください。
        # 一般的な「すべて同意する」ボタンを想定しています。
        cookie_button_xpath = "//button[contains(text(), 'Accept All')]"

        print("クッキー同意ボタンを探しています...")
        accept_button = wait.until(
            EC.element_to_be_clickable((By.XPATH, cookie_button_xpath))
        )

        print("同意ボタンをクリックします。")
        accept_button.click()

        # ポップアップが消えるのを少し待ちます
        time.sleep(1)

    except TimeoutException:
        # 10秒待ってもボタンが見つからなければ、表示されていないと判断して次に進みます
        print("クッキー同意ボタンは見つかりませんでした。処理を続行します。")

    # --- 【ステップ2】 目的のタブをJavaScriptでクリックする ---
    # 1. "Fund Flows" タブをクリック
    print("「Fund Flows」タブをクリックします...")
    # 要素がDOM上に読み込まれるのを待機
    fund_flows_tab = wait.until(
        EC.presence_of_element_located((By.ID, "fp-menu-fund-flows"))
    )
    # JavaScript を使ってクリックを実行
    driver.execute_script("arguments[0].click();", fund_flows_tab)

    # Fund Flowsタブのメインコンテンツが表示されるまで待機
    print("「Fund Flows」のコンテンツが表示されるのを待ちます...")
    wait.until(EC.visibility_of_element_located((By.ID, "fund-page-fund-flows")))
    print("コンテンツが表示されました。")
    # ---------
    # Fund Flowsタブでの操作
    # ---------

    time.sleep(2)

    # 2. "Overview" タブをクリック
    print("「Overview」タブをクリックします...")
    # 要素がDOM上に読み込まれるのを待機
    overview_tab = wait.until(
        EC.presence_of_element_located((By.ID, "fp-menu-overview"))
    )
    # JavaScript を使ってクリックを実行
    driver.execute_script("arguments[0].click();", overview_tab)
    # Overviewタブのメインコンテンツが表示されるまで待機
    print("「Overview」のコンテンツが表示されるのを待ちます...")
    wait.until(EC.visibility_of_element_located((By.ID, "fund-page-overview")))
    print("コンテンツが表示されました。")

    # ---------
    # Overviewタブでの操作
    # ---------

    print("タブの切り替えが完了しました。")

except Exception as e:
    print(f"エラーが発生しました: {e}")

finally:
    # 処理が終わったらブラウザを閉じる（必要に応じてコメントを外してください）
    print("処理が終了しました。5秒後にブラウザを閉じます。")
    time.sleep(5)
    driver.quit()


https://www.etf.com/VOO にアクセスします...
クッキー同意ボタンを探しています...
クッキー同意ボタンは見つかりませんでした。処理を続行します。
「Fund Flows」タブをクリックします...
「Fund Flows」のコンテンツが表示されるのを待ちます...
コンテンツが表示されました。
「Overview」タブをクリックします...
「Overview」のコンテンツが表示されるのを待ちます...
コンテンツが表示されました。
タブの切り替えが完了しました。
処理が終了しました。5秒後にブラウザを閉じます。


In [ ]:
def scrape_key_value_data(driver, wait, container_id: str) -> dict:
    """
    指定されたコンテナID内のキーと値のペアをスクレイピングして辞書として返す関数
    """
    data_dict = {}
    print(f"ID='{container_id}'のデータを待機しています...")

    # 1. データが表示されるまで待機
    # プレースホルダー（loading...）対策で、2段階の待機を行う
    wait.until(
        lambda d: len(d.find_elements(By.CSS_SELECTOR, f"#{container_id} .inline-flex"))
        > 0
    )
    try:
        wait.until(
            EC.invisibility_of_element_located(
                (
                    By.XPATH,
                    f"//*[@id='{container_id}']//*[contains(text(), 'loading...')]",
                )
            )
        )
    except:
        # loading... が最初から表示されない場合もあるため、タイムアウトは無視して続行
        pass

    print(f"ID='{container_id}'のデータを検出しました。")

    # 2. データを抽出
    container_div = driver.find_element(By.ID, container_id)
    data_rows = container_div.find_elements(By.CLASS_NAME, "inline-flex")
    for row in data_rows:
        children = row.find_elements(By.TAG_NAME, "div")
        if len(children) == 2:
            key = children[0].text
            value = children[1].text
            data_dict[key] = value

    return data_dict


# --- メインの処理 ---
ticker = "VOO"
url = f"https://www.etf.com/{ticker}"
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
wait = WebDriverWait(driver, 10)  # 待機時間を10秒に設定

try:
    # サイトにアクセス
    print(f"{url} にアクセスします...")
    driver.get(url)

    # クッキー同意ボタンの処理
    try:
        cookie_button_xpath = "//button[contains(text(), 'Accept All')]"
        print("クッキー同意ボタンを探しています...")
        accept_button = wait.until(
            EC.element_to_be_clickable((By.XPATH, cookie_button_xpath))
        )
        print("同意ボタンをクリックします。")
        accept_button.click()
        time.sleep(1)
    except TimeoutException:
        print("クッキー同意ボタンは見つかりませんでした。処理を続行します。")

    # "Overview" タブをクリック
    overview_tab = wait.until(
        EC.presence_of_element_located((By.ID, "fp-menu-overview"))
    )
    driver.execute_script("arguments[0].click();", overview_tab)

    # Overviewタブのメインコンテンツが表示されるまで待機
    wait.until(EC.visibility_of_element_located((By.ID, "fund-page-overview")))

    # 関数を呼び出して、それぞれのデータを取得
    summary_data = scrape_key_value_data(driver, wait, "summary-data")
    classification_data = scrape_key_value_data(
        driver, wait, "classification-index-data"
    )

    # 取得した2つの辞書を結合
    combined_data = {**summary_data, **classification_data}

    # 結合した辞書から最終的なDataFrameを作成
    df = pd.DataFrame([combined_data])
    df = df.assign(datetime_created=datetime.datetime.now(), Ticker=ticker)

    print("\n--- データフレームの作成に成功しました ---")
    display(df)

except Exception as e:
    print(f"処理中にエラーが発生しました: {e}")

finally:
    # 処理が終わったらブラウザを閉じる
    print("処理が終了しました。5秒後にブラウザを閉じます。")
    time.sleep(5)
    driver.quit()


https://www.etf.com/VOO にアクセスします...
クッキー同意ボタンを探しています...
クッキー同意ボタンは見つかりませんでした。処理を続行します。
「Fund Flows」タブをクリックします...
「Fund Flows」のコンテンツが表示されるのを待ちます...
コンテンツが表示されました。
「Overview」タブをクリックします...
「Overview」のコンテンツが表示されるのを待ちます...
コンテンツが表示されました。
ID='summary-data'のデータを待機しています...
ID='summary-data'のデータを検出しました。
ID='classification-index-data'のデータを待機しています...
ID='classification-index-data'のデータを検出しました。

--- データフレームの作成に成功しました ---


,Issuer,Inception Date,Expense Ratio,AUM,Index Tracked,Segment,Structure,Asset Class,Category,Focus,Niche,Region,Geography,Index Weighting Methodology,Index Selection Methodology,Segment Benchmark,datetime_stored,Ticker
0,Vanguard,09/07/10,0.03%,$751.49B,S&P 500,MSCI USA Large Cap,Open-Ended Fund,Equity,Size and Style,Large Cap,Broad-based,North America,U.S.,Market Cap,Committee,MSCI USA Large Cap,2025-09-12 18:11:30.743301,VOO


処理が終了しました。5秒後にブラウザを閉じます。


In [ ]:
def scrape_key_value_data(driver, wait, container_id: str) -> dict:
    """
    指定されたコンテナID内のキーと値のペアをスクレイピングして辞書として返す関数
    """
    data_dict = {}
    try:
        print(f"  ID='{container_id}'のデータを待機しています...")
        # 1. データが表示されるまで待機
        wait.until(
            lambda d: len(
                d.find_elements(By.CSS_SELECTOR, f"#{container_id} .inline-flex")
            )
            > 0
        )
        # loading... が消えるまで待機
        wait.until(
            EC.invisibility_of_element_located(
                (
                    By.XPATH,
                    f"//*[@id='{container_id}']//*[contains(text(), 'loading...')]",
                )
            )
        )
        print(f"  ID='{container_id}'のデータを検出しました。")
        # 2. データを抽出
        container_div = driver.find_element(By.ID, container_id)
        data_rows = container_div.find_elements(By.CLASS_NAME, "inline-flex")
        for row in data_rows:
            children = row.find_elements(By.TAG_NAME, "div")
            if len(children) == 2:
                key = children[0].text
                value = children[1].text
                if key and value:  # キーと値が両方存在する場合のみ追加
                    data_dict[key] = value
    except TimeoutException:
        print(
            f"  警告: ID='{container_id}'のデータが見つからなかったか、タイムアウトしました。"
        )
    except Exception as e:
        print(f"  エラー: ID='{container_id}'のデータ取得中にエラーが発生しました: {e}")
    return data_dict


def scrape_all_tickers(tickers: list) -> pd.DataFrame:
    """
    tickerのリストを受け取り、ループ処理でデータをスクレイピングして
    1つのDataFrameに結合して返すメイン関数。
    """
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service)
    wait = WebDriverWait(driver, 10)

    all_tickers_data = []

    try:
        # 最初に一度だけクッキー処理を試みる
        # どのページでも良いので、最初のtickerのページで実行
        initial_url = f"https://www.etf.com/{tickers[0]}"
        print(f"最初のURLにアクセスしてクッキー処理を行います: {initial_url}")
        driver.get(initial_url)
        cookie_wait = WebDriverWait(driver, 3)
        try:
            accept_button = cookie_wait.until(
                EC.element_to_be_clickable(
                    (By.XPATH, "//button[contains(text(), 'Accept All')]")
                )
            )
            print("クッキー同意ボタンをクリックします。")
            accept_button.click()
            time.sleep(1)
        except TimeoutException:
            print("クッキー同意ボタンは見つかりませんでした。")

        # tickerリストをループ処理
        for ticker in tickers:
            print(f"\n--- ticker '{ticker}' の処理を開始します ---")
            try:
                url = f"https://www.etf.com/{ticker}"
                print(f"{url} にアクセスします...")
                driver.get(url)

                # Overviewタブのメインコンテンツが表示されるまで待機
                print("  「Overview」のコンテンツが表示されるのを待ちます...")
                wait.until(
                    EC.visibility_of_element_located((By.ID, "fund-page-overview"))
                )
                print("  コンテンツが表示されました。")

                # データ取得
                summary_data = scrape_key_value_data(driver, wait, "summary-data")
                classification_data = scrape_key_value_data(
                    driver, wait, "classification-index-data"
                )

                if not summary_data and not classification_data:
                    print(
                        f"'{ticker}' のデータが取得できなかったため、スキップします。"
                    )
                    continue

                combined_data = {**summary_data, **classification_data}
                combined_data["Ticker"] = ticker
                combined_data["datetime_created"] = datetime.datetime.now()

                all_tickers_data.append(combined_data)
                print(f"--- ticker '{ticker}' の処理が正常に完了しました ---")

            except Exception as e:
                print(f"--- ticker '{ticker}' の処理中にエラーが発生しました: {e} ---")
                continue  # エラーが発生した場合は次のtickerへ

    finally:
        print("\n全ての処理が終了しました。ブラウザを閉じます。")
        driver.quit()

    # 取得した全データのリストからDataFrameを作成
    if not all_tickers_data:
        return pd.DataFrame()

    return pd.DataFrame(all_tickers_data)


db_path = ETF_DIR / "ETFDotCom.db"
conn = sqlite3.connect(db_path)
ticker_list = pd.read_sql("SELECT * FROM Ticker", con=conn)["Ticker"].unique().tolist()
conn.close()
display(ticker_list[:10])

final_df = scrape_all_tickers(ticker_list)

if not final_df.empty:
    print("\n--- 最終的な統合データフレーム ---")
    display(final_df)
else:
    print("\n有効なデータを取得できませんでした。")


OperationalError: unable to open database file

## Fund flow download


In [ ]:
def get_fund_flows_data(ticker: str) -> pd.DataFrame:
    """
    指定されたティッカーのFund Flowsデータを専用URLから取得し、DataFrameとして返す関数。
    """
    start_date = "1990-01-01"
    end_date = datetime.date.today().strftime("%Y-%m-%d")
    url = f"https://www.etf.com/etfanalytics/etf-fund-flows-tool-result?tickers={ticker},&startDate={start_date}&endDate={end_date}&frequency=DAILY"

    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service)
    wait = WebDriverWait(driver, 10)

    all_rows_data = []

    try:
        print(f"{url} にアクセスします...")
        driver.get(url)

        cookie_wait = WebDriverWait(driver, 3)
        try:
            cookie_button = cookie_wait.until(
                EC.element_to_be_clickable(
                    (By.XPATH, "//button[contains(text(), 'Accept All')]")
                )
            )
            print("クッキー同意ボタンをクリックします。")
            cookie_button.click()
            time.sleep(1)
        except TimeoutException:
            print("クッキー同意ボタンは見つかりませんでした。")

        # --- ★修正点: ステップ1 - テーブルのコンテナが表示されるまで待つ ---
        print("データテーブルが表示されるのを待ちます...")
        wait.until(EC.visibility_of_element_located((By.ID, "flow-detail")))

        # --- ★修正点: ステップ2 & 3 - ボタンまでスクロールしてクリック ---
        print("表示件数を100件に設定します...")
        display_100_button = wait.until(
            EC.presence_of_element_located((By.XPATH, "//button/span[text()='100']"))
        )

        # ボタンまでスクロール
        driver.execute_script(
            "arguments[0].scrollIntoView({block: 'center'});", display_100_button
        )
        time.sleep(1)  # スクロールのアニメーションを待つ

        # JavaScriptでクリック
        driver.execute_script("arguments[0].click();", display_100_button)

        print("テーブルが100件表示に更新されるのを待ちます...")
        time.sleep(3)

        # 全てのページをループしてデータを取得
        page_num = 1
        while True:
            print(f"{page_num}ページ目のデータを取得中...")
            table = wait.until(
                EC.visibility_of_element_located(
                    (By.ID, "fund-flow-result-output-table")
                )
            )
            rows = table.find_elements(By.XPATH, ".//tbody/tr")

            if not rows:
                print("テーブルに行が見つかりませんでした。")
                break

            for row in rows:
                cols = row.find_elements(By.TAG_NAME, "td")
                if len(cols) == 2:
                    date = cols[0].text
                    net_flows = cols[1].text
                    all_rows_data.append(
                        {"Date": date, f"{ticker} Net Flows": net_flows}
                    )

            try:
                next_button = driver.find_element(
                    By.XPATH, "//li[not(contains(@class, 'disabled'))]/a[text()='Next']"
                )
                driver.execute_script(
                    "arguments[0].scrollIntoView({block: 'center'});", next_button
                )
                time.sleep(1)
                driver.execute_script("arguments[0].click();", next_button)
                print("Nextボタンをクリックしました。")
                page_num += 1
                time.sleep(3)
            except NoSuchElementException:
                print("最終ページに到達しました。")
                break

    except Exception as e:
        print(f"処理中にエラーが発生しました: {e}")

    finally:
        driver.quit()

    if not all_rows_data:
        print("データが取得できませんでした。")
        return pd.DataFrame()

    df = pd.DataFrame(all_rows_data)
    df["Date"] = pd.to_datetime(df["Date"])
    df[f"{ticker} Net Flows"] = (
        df[f"{ticker} Net Flows"].str.replace(",", "").astype(float)
    )
    df = (
        df.rename(columns={"Date": "date", f"{ticker} Net Flows": "ETF Net Flows"})
        .assign(Ticker=ticker)
        .sort_values("date", ignore_index=True)
    )
    return df


# -------
ticker_symbol = "VOO"
fund_flows_df = get_fund_flows_data(ticker_symbol)

if not fund_flows_df.empty:
    print(f"\n--- {ticker_symbol} のファンドフローデータ取得に成功しました ---")
    print(f"取得したデータ件数: {len(fund_flows_df)}")
    display(fund_flows_df.head())  # 最初の5行を表示
    display(fund_flows_df.tail())  # 最後の5行を表示


https://www.etf.com/etfanalytics/etf-fund-flows-tool-result?tickers=VOO,&startDate=1990-01-01&endDate=2025-09-12&frequency=DAILY にアクセスします...
クッキー同意ボタンは見つかりませんでした。
データテーブルが表示されるのを待ちます...
表示件数を100件に設定します...
テーブルが100件表示に更新されるのを待ちます...
1ページ目のデータを取得中...
Nextボタンをクリックしました。
2ページ目のデータを取得中...
Nextボタンをクリックしました。
3ページ目のデータを取得中...
Nextボタンをクリックしました。
4ページ目のデータを取得中...
Nextボタンをクリックしました。
5ページ目のデータを取得中...
Nextボタンをクリックしました。
6ページ目のデータを取得中...
Nextボタンをクリックしました。
7ページ目のデータを取得中...
Nextボタンをクリックしました。
8ページ目のデータを取得中...
Nextボタンをクリックしました。
9ページ目のデータを取得中...
Nextボタンをクリックしました。
10ページ目のデータを取得中...
Nextボタンをクリックしました。
11ページ目のデータを取得中...
Nextボタンをクリックしました。
12ページ目のデータを取得中...
Nextボタンをクリックしました。
13ページ目のデータを取得中...
Nextボタンをクリックしました。
14ページ目のデータを取得中...
Nextボタンをクリックしました。
15ページ目のデータを取得中...
Nextボタンをクリックしました。
16ページ目のデータを取得中...
Nextボタンをクリックしました。
17ページ目のデータを取得中...
Nextボタンをクリックしました。
18ページ目のデータを取得中...
Nextボタンをクリックしました。
19ページ目のデータを取得中...
Nextボタンをクリックしました。
20ページ目のデータを取得中...
Nextボタンをクリックしました。
21ページ目のデータを取得中...
Nextボタンをクリックしました。
22ページ目のデータを取得中...
Nex

,Date,VOO Net Flows
0,2024-04-03,655.71
1,2024-04-02,348.66
2,2024-04-01,171.97
3,2024-03-29,0.00
4,2024-03-28,-150.81


,Date,VOO Net Flows
3911,2025-09-04,1232.82
3912,2025-09-05,1816.94
3913,2025-09-08,-104.61
3914,2025-09-09,3134.05
3915,2025-09-10,2787.59
